In [12]:
import pandas as pd
import wget
import os

if (not os.path.isfile("WineKMC.xlsx")):
    wget.download("http://blog.yhathq.com/static/misc/data/WineKMC.xlsx")

df_offers = pd.read_excel("WineKMC.xlsx", sheetname=0)
df_offers.columns = ["offer_id", "campaign", "varietal", "min_qty", "discount", "origin", "past_peak"]
df_offers.head()

,offer_id,campaign,varietal,min_qty,discount,origin,past_peak
0,1,January,Malbec,72,56,France,False
1,2,January,Pinot Noir,72,17,France,False
2,3,February,Espumante,144,32,Oregon,True
3,4,February,Champagne,72,48,France,True
4,5,February,Cabernet Sauvignon,144,44,New Zealand,True


In [2]:
df_transactions = pd.read_excel("/Users/bob/Development/WineKMC.xlsx", sheetname=1)
df_transactions.columns = ["customer_name", "offer_id"]
df_transactions['n'] = 1
df_transactions.head()

,customer_name,offer_id,n
0,Smith,2,1
1,Smith,24,1
2,Johnson,17,1
3,Johnson,24,1
4,Johnson,26,1


In [3]:
# join the offers and transactions table
df = pd.merge(df_offers, df_transactions)
# create a "pivot table" which will give us the number of times each 
# customer responded to a given variable
matrix = df.pivot_table(index=['customer_name'], columns=['offer_id'], values='n')
# a little tidying up. fill NA values with 0 and make the index into a column
matrix = matrix.fillna(0).reset_index()
x_cols = matrix.columns[1:]

In [4]:
from sklearn.cluster import KMeans

In [5]:
cluster = KMeans(n_clusters=5)
# slice matrix so we only include the 0/1 indicator columns in the clustering
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[2:]])
matrix.cluster.value_counts()

2    34
0    21
3    17
1    16
4    12
dtype: int64

In [6]:
from sklearn.decomposition import PCA

In [7]:
pca = PCA(n_components=2)
matrix['x'] = pca.fit_transform(matrix[x_cols])[:,0]
matrix['y'] = pca.fit_transform(matrix[x_cols])[:,1]
matrix = matrix.reset_index()

In [8]:
customer_clusters = matrix[['customer_name', 'cluster', 'x', 'y']]
customer_clusters.head()

offer_id,customer_name,cluster,x,y
0,Adams,3,-1.007580,0.108215
1,Allen,2,0.287539,0.044715
2,Anderson,1,0.392032,1.038391
3,Bailey,4,-0.699477,-0.022542
4,Baker,4,-0.088183,-0.471695


In [9]:
df = pd.merge(df_transactions, customer_clusters)
df = pd.merge(df_offers, df)

In [10]:
df['is_4'] = df.cluster==4
df.groupby("is_4").varietal.value_counts()

is_4                     
False  Champagne             79
       Pinot Noir            44
       Espumante             36
       Cabernet Sauvignon    32
       Prosecco              28
       Malbec                28
       Merlot                18
       Chardonnay            15
       Pinot Grigio          11
True   Prosecco              15
       Pinot Grigio           6
       Espumante              4
       Malbec                 4
       Merlot                 2
       Champagne              2
dtype: int64

In [11]:
df.groupby("is_4")[['min_qty', 'discount']].mean()

,min_qty,discount
is_4,,
False,62.948454,59.945017
True,18.363636,55.393939
